Name: [Serdar Metin]

I hereby declare that I observed the honour code of the university when preparing the homework.

## Solution to 16.3-5 and 18



## Pr?gr?mm?ng?H?m?w?rk 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print(T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


## Question 1

We define a function "randc(pmf)" which takes a probability mass function as a vector and calculates the corresponding cumulative mass function. By drawing uniformly random from [0,1] interval and finding the corresponding interval in the cmf vector(i.e the minimum value that exceeds the drawn random number), the function returns the index of the random number.

$T$ is converted into a numpy array, to be cast into float.

In the main loop we start with the stationary distribution (i.e. $x_0 = \text{'.'}$) and iterate $N$ times to draw with randc. In each iteration the index is converted into the character to be appended to the array, to be appended to the array, and the pmf function is updated with the relevant column in the table $T$.

In [3]:
import numpy as np
import csv
from IPython.display import display, Latex

def randc(pmf):
    cmf=np.cumsum(pmf)          # calculate cmf from pmf
    draw=np.random.rand()       # draw from [0-1], uniformly
    i=0
    while cmf[i]<draw:          # find the corresponding letter
        i+=1
    return i

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}
idx2letter = {i:c for i,c in enumerate(alphabet)}         # well ...



T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)
        
T=np.array(T)
T=T.astype(np.float)        # convert string to float


## main loop ##

N=15

randomstring=[]
current=26                      # index of the stationary distribution

for i in range(N):
    current=randc(T[current])
    randomstring.append(idx2letter[current])

print(randomstring)

['u', 't', 'h', 'a', 'y', 'o', '.', 'd', 's', 't', '.', 't', 'h', 'e', 'i']


## Question 2

The strategy we use is to isolate the missing letter sequences, and use the immediately adjacent letters in both ends to estimate the first missing letter of the sequence. The process is recursively applied to cover a missing letter sequence, with the newly added letter as the preceeding end of the remaining sequence.

The probability mass function of the target letter is conditioned on the joint distribution of the immediately preceeding letter and the final letter of the sequence $$p(x_1|x_0,x_n)$$, in a sequence of $n-2$ missing letters. Because of the Markov quality, the letters out of these boundaries have no effect on the distribution.

Depending on the independence assumption for the $x_0$ and $x_n$, we may rewrite: $$p(x_1|x_0)p(x_1|x_n)$$
The first term is easily obtained from the table $T$, by fixing the value of $x_0$. For the second term, we have to rewrite the formula as: $$\frac{p(x_n|x_1)p(x_1)}{p(x_n)}$$ The nominator can be expanded as: $$p(x_2|x_1)p(x_3|x_2)...p(x_n|x_{n-1})$$ Since each conditional probability term can be represented with multiplication with $T$, the term equals $T^n$. Although it is a matri power operation, it is an easy one since the matrix is Markov matrix and guranteed to have full rank and $S \lambda S^{-1}$ decomposition. Finally, we take the corresponding row for the fixed $x_n$ and normalize it with its own norm to obtain a conditioned probability distribution.

We define $\phi(\vec v, \vec u) = \vec w$, where $$\vec w_i= \vec v_i . \vec u_i$$ and $Z$ as $$Z=||\vec w ||$$

We draw from the resulting distribution represented in $\vec w$. Below is the code:

In [191]:
import numpy as np
import csv
from IPython.display import display, Latex

def randc(pmf):
    cmf=np.cumsum(pmf)          # calculate cmf from pmf
    draw=np.random.rand()       # draw from [0-1], uniformly
    i=0
    while cmf[i]<draw:          # find the corresponding letter
        i+=1
    return i

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}
idx2letter = {i:c for i,c in enumerate(alphabet)}         # well ...

test_strings = [list('th__br__n.f_x.'),list('_u_st__n_.to_be._nsw_r__'),list('i__at_._a_h_n_._e_r_i_g'),list('q___t.___z._____t.__.___.__.')]

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)
T=np.array(T)
T=T.astype(np.float)


def choose(begin, end, length):                         # function for finding the target letter
    prev=T[begin]
    if(length==0):
        pmf=prev
    else:
        nex=np.linalg.matrix_power(T,length).transpose()[end]
        pmf=nex*prev
        pmf/=np.sum(pmf)
    return randc(pmf)

def find(i, j):
    length=0
    while(test_strings[i][j+length]=='_' and j+length<len(test_strings[i])-1):
        length+=1
    if(j==0):
        begin=26
    else:
        begin=letter2idx[test_strings[i][j-1]]
    if(j+length==len(test_strings[i])-1):       
        end=26
    else:
        end=letter2idx[test_strings[i][j+length]]
    ch=idx2letter[choose(begin, end, length)]
    test_strings[i][j]=ch
    return

## main loop ##

for i, str  in enumerate(test_strings):
    for j, ch in enumerate(str):
        if(ch == '_'):
            find(i, j)
for str in test_strings:
    print(''.join(str))

the.brnan.fex.
oussthand.to.be.answhrde
intat..hachent.wegr.isg
qurit.terz.ple.ot.be.m.t.pt.


## Question 3

The same procedure is done with taking the maximal element of $\vec w$

In [185]:
import numpy as np
import csv
from IPython.display import display, Latex

alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}
idx2letter = {i:c for i,c in enumerate(alphabet)}         # well ...

test_strings = [list('th__br__n.f_x.'),list('_u_st__n_.to_be._nsw_r__'),list('i__at_._a_h_n_._e_r_i_g'),list('q___t.___z._____t.__.___.__.')]

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)
T=np.array(T)
T=T.astype(np.float)


def choose(begin, end, length):                         # function for finding the target letter
    prev=T[begin]
    if(length==0):
        pmf=prev
    else:
        nex=np.linalg.matrix_power(T,length).transpose()[end]
        pmf=nex*prev
        pmf/=np.sum(pmf)
    return np.argmax(pmf)

def find(i, j):
    length=0
    while(test_strings[i][j+length]=='_' and j+length<len(test_strings[i])-1):
        length+=1
    if(j==0):
        begin=26
    else:
        begin=letter2idx[test_strings[i][j-1]]
    if(j+length==len(test_strings[i])-1):       
        end=26
    else:
        end=letter2idx[test_strings[i][j+length]]
    ch=idx2letter[choose(begin, end, length)]
    test_strings[i][j]=ch
    return

## main loop ##

for i, str  in enumerate(test_strings):
    for j, ch in enumerate(str):
        if(ch == '_'):
            find(i, j)
for str in test_strings:
    print(''.join(str))

the.br.an.fex.
oursthend.to.be.answere.
in.ath.wathend.he.r.ing
qur.t.thiz.the.at.an.the.an.


## Question 4

The search process may be started from anywhere of the missing sequence, so instead of in order filling, it is possible to fill the missing letters in another fashion. The first candidate may be a procedure similar to binary search. Namely, the mid point of the missing letter sequence is guessed first and the process is repeated recursively.

An alternative model might be a Markov 2 process, where the distribution of the letters are conditioned on not only the immediate predecessors but also to their immediate predecessors also. Unfortunately, the elaboration of the computing process is not guaranteed to yield better results. It is more of a function of the genuine distribution (i.e. how the thing really is distributed).